<a href="https://colab.research.google.com/github/rovargasc/curso_IA/blob/main/Tarea1_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tarea 1: Replicación de las redes mostradas en el vídeo de Sotaquira


### Usando la CPU

In [1]:
## Librerías necesarias
import numpy as np
import os
import gzip

In [2]:
def load_mnist(ruta, tipo = "train"):
  ruta_categorias = os.path.join(ruta, "%s-labels-idx1-ubyte.gz" % tipo)
  ruta_imagenes = os.path.join(ruta, "%s-images-idx3-ubyte.gz"% tipo)

  with gzip.open(ruta_categorias, "rb") as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype = np.uint8, offset = 8)

  with gzip.open(ruta_imagenes, "rb") as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas

In [3]:
# Estas líneas las dejé comentadas porque no cargué la base de datos así.
# Bajé los archivos de keras

#from google.colab import drive
#drive.mount('/content/gdrive')

#ruta = 'gdrive/My Drive/Colab Notebooks/fashion_mnist_data'

#X_train, Y_train = load_mnist(ruta, tipo='train')
#X_test, Y_test = load_mnist(ruta, tipo='test')

In [4]:
# Aquí bajo los datos de keras
from keras.datasets import mnist  

In [ ]:
# The MNIST data is split between 60,000 28 x 28 pixel training images and 10,000 28 x 28 pixel images
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()


11501568/11490434 [==============================] - 0s 0us/step


## Ajuste de los datos

In [ ]:
# Se debe garantizar que el tamaño del dataset sea un múltiplo de 128 , pero esto es para la TPU.  IMPORTANTE.


X_train = X_train[0:59904,:]  #El número más cercano a 60000 y que es a la vez múltiplo de 128 es 59904,
X_test = X_test[0:9984,:]    # el set de validación tiene originalmente 10000 datos, y lo reajustaremos a un tamaño de 9984.
Y_train = Y_train[0:59904]   # El número más cercano a 60000 y que es a la vez múltiplo de 128 es 59904,
Y_test = Y_test[0:9984]     # el set de validación tiene originalmente 10000 datos, y lo reajustaremos a un tamaño de 9984.

X_train = np.reshape(X_train,(59904,28,28,1)) # con esto se garantiza que cada dato es una imagen de grises de 28 x 28
X_test = np.reshape(X_test,(9984,28,28,1))  # con esto se garantiza que cada dato es una imagen de grises de 28 x 28

### Puesta en marcha del modelo convolucional

In [ ]:
%tensorflow_version 2.x   # Para garantizar que la versión 2.x sea importada
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__)
tf.random.set_seed(200)  #  garantizo que los valores iniciales sean aleatorios pero los mismos tanto para CPU,GPU y TPU

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x   # Para garantizar que la versión 2.x sea importada`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Versión de TensorFlow: 2.8.0


### Implementación del modelo

In [ ]:
# Primero se hace una normalización (BatchNormalization) para facilitar la
# convergencia del modelo. Luego se agregan los filtros convolucionales.
# Luego se agrega una capa de max pooling
# Luego una capa de dropout para reducir el overfitting del modelo.
# La diferencia entre las 3 capas radica en el número de filtros convolucionales
# usados: 64, 128 y 256

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64,(5,5), padding = "same", activation = "elu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128,(5,5), padding = "same", activation = "elu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256,(5,5), padding = "same", activation = "elu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))
model.add(tf.keras.layers.Dropout(0.25))

#### Aplanamiento de los datos, red neuronal con 256 neuronas y función de activación

In [ ]:
model.add(tf.keras.layers.Flatten()) # Aplanamiento
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation("elu"))
# Salida
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation("softmax"))

In [ ]:
## Ahora, antes del ENTRENAMIENTO, compilaremos el modelo. Es decir,
# vamos a definir el optimizador a usar (Adam)
# la función de error (entropía cruzada)
# y la métrica de desempeño (la precisión)

model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 28, 28, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                        

In [ ]:
### Se usará la función timeit para almacenar en una variable el tiempo promedio de cada iteración

import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    ## Este es el verdadero modelo. 
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

Epoch 1/2
287/468 [=================>............] - ETA: 4:57 - loss: 0.4466 - accuracy: 0.8887

KeyboardInterrupt: ignored

In [ ]:
print('Tiempo de entrenamiento ' + str(cpu_time)+ ' segundos')

### Trabajando con GPU

# Como paso previo, se irá a "Entorno de ejecución", luego click en cambiar tipo de entorno de ejecución, luego elegir la opción GPU

In [ ]:
# Se verificará la disponibilidad de la GPU con tf.test
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))

In [ ]:
import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')

### Ahora con la TPU

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # Detectar TPU
  print('TPU encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada!')

In [ ]:
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strategy.scope():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()

  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
import timeit

x_train = X_train.astype(np.float32)
y_train = Y_train.astype(np.float32)
x_test = X_test.astype(np.float32)
y_test = Y_test.astype(np.float32)

def entrenamiento_tpu():
  model.fit(x_train,y_train,validation_data=(x_test,y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')